Imports!

In [ ]:
%matplotlib inline

import h5py

from keras.models import Sequential, Graph, model_from_json
from keras.layers.core import Flatten, Dense
from keras.layers.convolutional import Convolution2D
from keras.optimizers import SGD
from keras.utils.visualize_util import to_graph

from IPython.display import SVG

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb

from scipy.ndimage.interpolation import rotate
from scipy.io import loadmat

from pprint import pprint
import copy

import train
import evaluate
from train import infer_sizes
import models

Now load up our data H5 and grab some trained weights for our model.

In [ ]:
# Load data and get a model
train_h5_path = '../cache/train-patches-mpii/samples-000001.h5'
train_neg_h5_path = '../cache/train-patches-mpii/negatives.h5'
val_h5_path = '../cache/val-patches-mpii/samples-000001.h5'
val_neg_h5_path = '../cache/val-patches-mpii/negatives.h5'
train_h5 = h5py.File(train_h5_path, 'r')
train_neg_h5 = h5py.File(train_neg_h5_path, 'r')
val_h5 = h5py.File(val_h5_path, 'r')
val_neg_h5 = h5py.File(val_neg_h5_path, 'r')

In [ ]:
train_images, train_flow = train_h5['images'], train_h5['flow']
train_neg_images, train_neg_flow = train_neg_h5['images'], train_neg_h5['flow']
val_images, val_flow = val_h5['images'], val_h5['flow']
val_neg_images, val_neg_flow = val_neg_h5['images'], val_neg_h5['flow']
ds_shape = infer_sizes(train_h5_path)

In [ ]:
sgd = SGD(lr=0.0001, nesterov=True, momentum=0.9)
model = models.vggnet16_joint_reg_class_flow(ds_shape, sgd, 'glorot_normal')
model.load_weights('../cache/kcnn-flow-rgb-tripose-from-3840-plus-1024/model-iter-10240-r181250.h5')

## Visualising network architecture

Start by doing a basic visualisation of our model and inspecting the shape of our data.

In [ ]:
print '# Data shapes'
print 'images:', train_images.shape
print 'flow:', train_flow.shape
print 'validation images:', val_images.shape
print 'validation flow:', val_flow.shape
print
print '# Network'
SVG(to_graph(model, show_shape=True).create(prog='dot', format='svg'))

In [ ]:
upgraded_model = models.upgrade_multipath_vggnet(model)
upgraded_model.compile(sgd, model.loss)

In [ ]:
SVG(to_graph(upgraded_model, show_shape=True).create(prog='dot', format='svg'))

### Saving and reloading models

Need to check that my convert to fully convolutional network --> save model to JSON and weights to HDF5 --> reload model from Matlab pipeline works.

In [ ]:
mod_json = upgraded_model.to_json()
with open('../cache/cnn_model.json', 'w') as fp:
    fp.write(mod_json)
upgraded_model.save_weights('../cache/cnn_model.h5')

In [ ]:
with open('../cache/cnn_model.json') as fp:
    json_data = fp.read()
m2 = model_from_json(json_data)
m2.load_weights('../cache/cnn_model.h5')

## Visualising the training set (no predictions)

Now write some functions to look at our data and also a few utilities for doing forward prop. These will be useful for inspecting activations and gradients, as well as verifying that I've written what I wanted to write to the file.

Note that some of these images will look weird because they've been padded (where necessary) with their edge pixel values. This is true of the flow as well.

In [ ]:
centroids = loadmat('../cache/centroids.mat')['centroids'][0].tolist()

def _label_to_coords(label):
    return label.reshape((-1, 2))

def get_centroids(classes, centroids):
    """Grab the centroid of the cluster associated with each classifier output.
    
    :param classes: One-of-K vector (N-first, K-second) indicating appropriate
                    centroids
    :param centroids: list of P elements, each of which is a CxJp array, where
                      P is the number of poselets, C is the number of clusters
                      per poselet, and Jp is the number of regressor outputs per
                      cluster
    :returns: N-element list of the form [(class, poselet, centroid)], where
              class is in [0, P] (0 is background), centroid is None (for
              background) or a Jp/2x2 array giving (x, y) locations for each
              joint, and poselet is a poselet index in [0, C)"""
    assert classes.ndim == 2, "Expect one-of-K classes!"
    class_nums = np.argmax(classes, axis=1)
    classes_per_poselet = (classes.shape[1] - 1) / len(centroids)
    rv = []
    
    for num in class_nums:
        if num == 0:
            rv.append((0, None, None))
            continue
        poselet_class_idx = (num - 1) % classes_per_poselet
        poselet = (num - 1) // classes_per_poselet
        centroid = centroids[poselet][poselet_class_idx]
        # Will have to fix this later, if I change number of
        # points per poselet
        assert centroid.shape == (16,)
        rv.append((
            poselet + 1, poselet_class_idx, _label_to_coords(centroid)
        ))

    return rv

In [ ]:
def _reshape_im(im):
    # images are stored channels-first, but numpy expects
    # channels-last
    return np.transpose(im, axes=(1, 2, 0))

def _vis_flow(flow):
    # clean visualisation of flow with angle of movement as
    # hue, magnitude as saturation and a constant V of 1
    x, y = flow
    # normed-log makes things stand out quite a bit
    mags = np.log(np.sqrt(x**2 + y**2) + 1)
    norm_mags = mags / max(mags.flatten())
    angles = (np.arctan2(x, y) + np.pi) / (2 * np.pi)
    ones = np.ones_like(angles)
    hsv = np.stack((angles, norm_mags, ones), axis=2)
    return hsv_to_rgb(hsv)

def _plot_coords(coords):
    # plot a label corresponding to a flattened joint vector
    for idx, coord in enumerate(coords):
        plt.plot(coord[0], coord[1], marker='+')
        plt.text(coord[0], coord[1], str(idx))

def show_datum(image, flow, label=None):
    # First frame
    im1 = _reshape_im(image[:3])
    plt.subplot(131)
    plt.imshow(im1)
    plt.axis('off')
    plt.text(-10, -10, 'frame1')
    
    if label is not None:
        coords = _label_to_coords(label)
        first_coords = coords[:len(coords)//2]
        _plot_coords(first_coords)
    
    # Second frame
    im2 = _reshape_im(image[3:6])
    plt.subplot(132)
    plt.imshow(im2)
    plt.axis('off')
    plt.text(-10, -10, 'frame2')
    
    if label is not None:
        second_coords = coords[len(coords)//2:]
        _plot_coords(second_coords)
    
    # Optical flow
    if flow is not None:
        im_flow = _vis_flow(flow)
        plt.subplot(133)
        plt.imshow(im_flow)
        plt.axis('off')
        plt.text(-10, -10, 'flow')

    plt.show()
    
def get_joints(fp, index, ds_order=('left', 'right', 'head')):
    class_num = np.argmax(fp['class'][index])
    ds_name = ds_order[(class_num-1)%3]
    return fp[ds_name][index]

for i in np.random.permutation(len(train_images))[:0]:
    # Just visualise the input data so that I know I'm writing it out correctly
    print 'Training ground truth (NOT prediction)', i
    j = get_joints(train_h5, i)
    show_datum(train_images[i], train_flow[i], j)
    for ds in ('left', 'right', 'head'):
        jx = train_h5[ds][i]
        print('{}: {}'.format(ds, jx))
    print('Class: {}'.format(train_h5['class'][i]))
    
for i in np.random.permutation(len(train_neg_images))[:0]:
    # Just visualise the input data so that I know I'm writing it out correctly
    print 'Training negative', i
    show_datum(train_neg_images[i], train_neg_flow[i])
    for ds in ('left', 'right', 'head'):
        jx = train_neg_h5[ds][i]
        print(jx.shape)
        # print('{}: {}'.format(ds, jx))
    print('Class: {}'.format(train_neg_h5['class'][i]))
    
# for i in np.random.permutation(len(val_joints))[:3]:
#     print 'Validation ground truth (NOT prediction)', i
#     show_datum(val_images[i], val_flow[i], val_joints[i])

# for i in np.random.permutation(len(val_neg_images))[:3]:
#     print 'Validation negative', i
#     show_datum(val_neg_images[i], val_neg_flow[i])

train_centroid_classes = train_h5['poselet']
train_centroids = get_centroids(train_centroid_classes[:], centroids)
for i in np.random.permutation(len(train_images))[:100]:
    print 'Training ground truth poselet (NOT prediction)', i
    cls, pslt, coords = train_centroids[i]
    show_datum(train_images[i], train_flow[i], coords)
    true_cls = np.argmax(train_h5['class'][i])
    assert true_cls == cls, '%i (true) vs. %i (from gc)' % (true_cls, cls)
    print('Class: {}, poselet: {}\n\n\n'.format(true_cls, pslt))

## Joint regressor results

This regressor takes in both flow and RGB data, pushes them through different streams, then outputs predictions for each of 3 poselets, along with a confidence for which poselet is in the frame.

## Results

Now we can try evaluating the CNN on some of our training and evaluation data, just to see whether it's learning anything useful.

In [ ]:
def evaluate_on_datum(data, model=model):
    mps = train.read_mean_pixels('../cache/mean_pixel.mat')
    data = train.sub_mean_pixels(mps, data)
    return model.predict(data)

In [ ]:
def evaluate_random_samples(images, flow, true_classes, num_samples, title='Sample'):
    for i in np.random.permutation(len(images))[:num_samples]:
        print('\n\n\n{} {}'.format(title, i))
        
        # Evaluate
        preds = evaluate_on_datum({
            'images': images[i:i+1], 'flow': flow[i:i+1]
        })
        
        # Get class info
        class_names = ('background', 'left', 'right', 'head')
        tc_idx = np.argmax(true_classes[i])
        out_probs = preds['class'][0]
        pc_idx = np.argmax(preds['class'][0])
        pc_prob = out_probs[pc_idx] * 100
        print('Class confidences: {}'.format(preds['class'][0]))
        print('True class: {}; Predicted class: {} ({}%)'.format(
                class_names[tc_idx],
                class_names[pc_idx], pc_prob
        ))
        print(u'\u2713 Correct class' if pc_idx == tc_idx
              else u'\u2717 Incorrect class')
        
        # Visualise
        if tc_idx > 0:
            label = preds[class_names[tc_idx]]
        else:
            label = None
        show_datum(images[i], flow[i], label=label)
        
        # Get error
        pos_mask = true_classes[i].astype('bool')
        cross_entropy = -np.log(out_probs[pos_mask]).sum() - np.log(out_probs[~pos_mask]).sum()
        tc_name = class_names[tc_idx]
        l1_dist = preds[class_names[tc_idx]]

print('# Validation images')
evaluate_random_samples(val_images, val_flow, val_h5['class'], 100, title='Validation datum')
    
print('\n\n\n# Training images')
evaluate_random_samples(train_images, train_flow, train_h5['class'], 100, title='Training datum')

# These are much less interesting because the classifier is good at picking out background patches.t
print('\n\n\n# Validation negatives')
evaluate_random_samples(val_neg_images, val_neg_flow, val_neg_h5['class'], 20, title='Validation negative')

## Measuring accuracy

### RGB-only model (PCP, pixel threshold accuracy)

In [ ]:
def plot_acc(thresholds, accs, plot_title, labels):
    if not isinstance(accs, list) and accs.ndim == 1:
        accs_list = [accs]
        label_list = [labels]
    else:
        accs_list = np.array(accs).T
        label_list = labels
    for acc, label in zip(accs_list, label_list):
        plt.plot(thresholds, acc, label=label)
    plt.ylim((0, 1))
    plt.xlim((min(thresholds), max(thresholds)))
    plt.ylabel('accuracy')
    plt.xlabel('threshold (px)')
    plt.title(plot_title)
    plt.legend()
    plt.show()

test_in_vals = np.linspace(0, 50, 100)
plot_acc(test_in_vals, -1 / (test_in_vals + 1) + 1, 'Example plot', 'foo')

In [ ]:
mp_path = '../cache/mean_pixel.mat'
all_predictions = evaluate.get_predictions(rgb_model, mp_path, images=val_images, batch_size=48)

In [ ]:
labels = [
    'shoulder 1',
    'elbow 1',
    'wrist 1',
    'shoulder 2',
    'elbow 2',
    'wrist 2'
]
all_gts = evaluate.label_to_coords(np.array(val_joints))
thresholds = np.linspace(0, 70, 100)
accs = evaluate.score_predictions_acc(all_gts, all_predictions, thresholds)
plot_acc(thresholds, accs, 'RGB model accuracy (all joints)', labels)

In [ ]:
limbs = [
    (0, 1),
    (1, 2),
    (3, 4),
    (4, 5)
]
limb_names = [
    'uarm1',
    'farm1',
    'uarm2',
    'farm2'
]

pcps = evaluate.score_predictions_pcp(all_gts, all_predictions, limbs)

def show_pcp(pcps, limb_names):
    fmt = lambda s: '{:>20}'.format(s)
    print(fmt('limb name') + ''.join(fmt(l) for l in limb_names))
    print(fmt('pcp') + ''.join(fmt(p) for p in pcps))
    
show_pcp(pcps, limb_names)

### Flow-only model (pixel threshold accuracy)

In [ ]:
flow_predictions = evaluate.get_predictions(flow_model, mp_path, flows=val_flow, batch_size=48)

In [ ]:
flow_labels = [
    'wrist 1',
    'wrist 2'
]
# There's no PCP here because we only have disconnected joints
flow_accs = evaluate.score_predictions_acc(all_gts[:, (2, 5)], flow_predictions, thresholds)
plot_acc(thresholds, flow_accs, 'Flow model accuracy (wrist only)', flow_labels)